In [1]:
# !/usr/bin/env python3

#cd ~/Desktop/snet/gits/lang-learn-repo/alex_tests

import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import pylab
import sys  
import subprocess
import pywt as pywt
import itertools

from sklearn import metrics
from sklearn import datasets
from sklearn.cluster import KMeans
from sklearn.metrics import pairwise_distances
from sklearn.decomposition import PCA

from mpl_toolkits import mplot3d

limit=10
runPredictor=True
anomalyType = 0 #0 es all, 1 es exclusive

In [2]:
#def PCAmapper(s,data):

#def getCleanGrammar(s):

#def getRules(s,word):

#def getPossibleNextWords: 

#def main:
#  for t in timeStream:
#    read data at time t
#    group data by clusters
#
#    for s in signalClusters:
#      map data into the pca normaliced space used to 'train' the 'model' (need to see how to do this)
#      symbolize based on the location in the pca space
#      get the index used in the grammar file to refer to this word
#      filter only words that correspond to an anomaly
#      readGrammarFile of s
#      process the file to have each word and its rules in a easy to use format (need to see how to do this)
#
#      #this depends on how the grammar works, and i will ask clarification of that with andres
#      for a in anomalousWord:
#        find it in the processedGrammarFile
#        if t = 0:
#          get all the disjoints for that word
#          initialize disjoints with position 0
#        else:
#          for d in disjoints:
#            check if current word would be at position p of the disjoint
#            if it is:
#              position = position+1
#            else:
#              position = 0
#            
#            if position == maxPosition:
#              issue warning

In [3]:
def ruleToPhraseSets(rules):
    words = []
    for r in rules:
        r = r.replace("(","")
        r = r.replace(")","")
        new = r.split(" & ")
        words = words + [new]
    words = list(words for words,_ in itertools.groupby(words))
    
    words2 = []
    for w in words:
        w2 = list(set(w))
        w2.sort()
        words2 = words2 + [w2]
    
    #print(words)
    #print(words2)
    return words2

In [4]:
#- es despues de (o que el otro lo tiene a la izquierda)
#+ es andes de (o que el otro lo tiene al a derecha)
def phrasesFromSets(desc,sets):
    size = len(desc[0])
    phrases = []
    for s in sets:
        if len(s)==1:
            if(s[0][-1]=='-'):
                temp = s[0]
                temp = temp.replace('-','')
                temp = temp.replace(desc[0],'',1)
                phrase = temp + desc[0]
            elif(s[0][-1]=='+'):
                temp = s[0]
                temp = temp.replace('+','')
                temp = temp.replace(desc[0],'',1)
                phrase = desc[0] + temp
            else:
                print("error, no sign")
        else:
            phrase = ""
            unjoined = []
            count = len(s)
            for p in s:
                if(p[-1]=='-'):
                    temp = p[:-1]
                    temp = temp.replace(desc[0],'',1)
                    sub = temp + desc[0]
                elif(p[-1]=='+'):
                    temp = p[:-1]
                    temp = temp.replace(desc[0],'',1)
                    sub = desc[0] + temp
                else:
                    print("error, no sign")
                    
                if(count<len(s)):
                    if (phrase[:size]==sub[-size:]):
                        phrase = sub + phrase[size:]
                    elif (sub[:size]==phrase[-size:]):
                        phrase = phrase[:size] + sub
                    else:
                        #-----------#revisar importancia el orden #--------------------------#
                        if(p[-1]=='-'):
                            phrase = sub + phrase[size:]
                            
                        elif(p[-1]=='+'):
                            phrase = phrase[:size] + sub
                            
                        #unjoined = unjoined + [sub]
                        #print(s,"error, no join")
                        #-----------#revisar importancia el orden #--------------------------#
                else:
                    phrase = sub
                    
                count = count-1
        phrases = phrases + [phrase]
    return phrases

In [5]:
def getCleanGrammar(s):  
    print(s)
    with open(s, 'r') as f:
        data = f.readlines()
 
    del data[0]
    del data[0]
    del data[0]
    del data[0]
    del data[-1]
    del data[-1]
    del data[-1]
    del data[-1]
    
    full = len(data)
    for r in range(full):
        line = data[full-r-1]
        if(len(line)==1):
            del data[full-r-1]
        elif(line[0]=="%"):
            data[full-r-1] = line[2:-1]
        elif(line[0]=="\""):
            data[full-r-1] = line[1:-3]
        else:
            data[full-r-1] = line[:-2].split(" or ")
    
    clean = []
    mapper = {}
    for r in range(0,len(data),3): 
        joined = [data[r],data[r+1]]
        mapper[joined[1]]=joined[0]
        phraseSets = ruleToPhraseSets(data[r+2])
        phrases = phrasesFromSets(joined,phraseSets)
        joined = joined + [phrases]
        clean = clean + [joined]
        
    return mapper,clean

In [6]:
def getBadRules(clean,maps,keys,exAnomaly,limit,probRisk):
    size = len(clean[0][0])
    maps2 = {}
    for m in maps:
        maps2[maps[m]]=m
    
    
    clean2 = {}
    for c in clean:
        clean2[c[0]]=c[2]
    
    badRules = []
    for rule in clean:
        if rule[1] in exAnomaly:
            badRules = badRules + rule[2]
            
    badRules = list(set(badRules))
    badRules.sort()
    badRuleWord = []
    badRuleCounter = []
    badRuleWordPos = []
    badRuleWordR = []
    badRuleCounterR = []
    badRuleWordPosR = []
       
    r=0
    stop = False
    newRules = badRules
    tester = 1
    while tester != 0:
        badRules = list(set(badRules + newRules))
        badRules.sort()
        newRules = []
        stop = True
        
        tester = len(badRules)
        for rule in badRules:
            if maps2[rule[:size]] in exAnomaly:
                #la primera palabra es anomalia, no hay que agregarle
                tester = tester-1
            elif len(rule)/size >= limit:
                #ya esta muy grande la oracion
                tester = tester-1
            else:
                subRules = []
                for c in clean2[rule[:size]]:
                    if rule[:size]==c[-size:]:
                        subRules = subRules + [c[:-size]+rule]
                
                if(len(subRules)>0):
                    badRules.remove(rule)
                    newRules = newRules + subRules
                else:
                    tester = tester-1
    
    for rule in badRules:
        pos1 = 999
        add1 = ""
        pos2 = 999
        add2 = ""
        risklevel = 0
        for anomaly in exAnomaly:
            loc = rule.find(maps[anomaly])
            if (loc >= 0 and loc < pos1):
                pos1 = loc
                add1 = maps[anomaly]
            prebadword = anomaly
            if (loc >= 0 and probRisk[exAnomaly.index(prebadword)]>risklevel):
                pos2 = loc
                add2 = maps[anomaly]
                risklevel = probRisk[tolerance.index(prebadword)]
        badRuleWord = badRuleWord + [add1]       
        badRuleWordPos = badRuleWordPos + [pos1]
        badRuleCounter = badRuleCounter + [0]
        badRuleWordR = badRuleWordR + [add2]       
        badRuleWordPosR = badRuleWordPosR + [pos2]
        badRuleCounterR = badRuleCounterR + [0]
        r=r+1
        
    print(len(badRules))
    return badRules, badRuleWord, badRuleCounter, badRuleWordPos, badRuleWordR, badRuleCounterR, badRuleWordPosR
    

In [7]:
path = '/home/alexcasar/Desktop/snet/gits/lang-learn-repo/alex_tests/data'
filenames = ['data11']
filename = filenames[0]
doc = pd.read_csv(path+"/"+filename+"/"+filename+".csv") 

grouped = pd.read_csv(path+"/"+filename+"/"+filename+"_PCA_3_3.csv") 
grouped['anomaly']=list(doc['anomaly'])
groupedAno = grouped[grouped['anomaly']==1]
groupedAnoN = grouped[grouped['anomaly']==0]


signalClusters=['sym0','sym1','sym2','sym3','sym4','sym5','sym6']
for s in signalClusters:
    print(s)
    langFile = path+"/"+filename+"/lang/"+s+"_PCA_3_3/dict.txt"
    maps, clean = getCleanGrammar(langFile)
    maps2 = {}
    for m in maps:
        maps2[maps[m]]=m
    keys = list(maps.keys())
    size = len(grouped[s][0])
    
    anomalies = list(groupedAno.groupby(by=s).count().iloc[:,0].index)
    nonanos = list(groupedAnoN.groupby(by=s).count().iloc[:,0].index)
    exAnomaly = []
    both = []
    for a in anomalies:
        if a not in nonanos:
            exAnomaly = exAnomaly + [a]
        else:
            both = both + [a]
    
    #choose from anomalies=0, exAnomaly=1 
    tols = ['all','exclusive']
    tolId = tols[anomalyType]
    
    probRisk = []
    if(tolId == 'all'):
        tolerance = anomalies
        for t in tolerance:
            ano = list(anomalies).count(t)
            nano = list(nonanos).count(t)
            certainty = 100*(ano/(ano+nano))
            probRisk = probRisk + [certainty]
    else:
        tolerance = exAnomaly
        certainty = 100
        for t in tolerance:
            probRisk = probRisk + [certainty]
        
    badRules, badRuleWord, badRuleCounter, badRuleWordPos, badRuleWordR, badRuleCounterR, badRuleWordPosR = getBadRules(clean,maps,keys,tolerance,limit,probRisk)
    
    rrule = []
    for w in grouped[s]:
        rrule = rrule + [maps[w]]
    fulllist = ''.join(list(grouped[s]))
    fullwordlist = ''.join(rrule)
    
    if(runPredictor):
        a=0
        print(tolerance)
        for word in grouped[s]:
            risk = 0
            a=a+1
                       
            if word in tolerance:
                certainty = round(probRisk[tolerance.index(word)])
                if check:
                    badRuleCounter = [-1]*len(badRuleCounter)
                check = False
                if (certainty == 100):
                    risk = 2
                    print(a,word,"ERR: Anomalous state",word,'currently active')
                else:
                    risk = 1
                    print(a,word,"WHI: State",word,"with",certainty,'% chance of being anomaouls, currently active')
                
            r=0
            minW = 999
            pastword = badRules[0]
            risklevel = 0
            for rule in badRules:
                if maps[word] in rule:                    
                    if risk == 0:
                        if badRuleCounter[r] == -1 or pastword != word:
                            badRuleCounter[r] = rule.rfind(maps[word])-1
                        elif badRuleCounter[r] < badRuleWordPos[r]:
                            badRuleCounter[r] = badRuleCounter[r] + 1
                        pos = badRuleWordPos[r] - badRuleCounter[r]
                        
                        if badRuleCounter[r] <= badRuleWordPos[r]:
                            minW = pos+1
                            badword = maps2[badRuleWord[r]]
                            badrule = rule
                    elif risk == 1:
                        if badRuleCounterR[r] == -1 or pastword != word:
                            badRuleCounterR[r] = rule.rfind(maps[word])-1
                        elif badRuleCounterR[r] < badRuleWordPosR[r]:
                            badRuleCounterR[r] = badRuleCounterR[r] + 1
                        pos = badRuleWordPosR[r] - badRuleCounterR[r]
                        prebadword = maps2[badRuleWordR[r]]
                        
                        if probRisk[tolerance.index(prebadword)]>risklevel:
                            minW = pos+1
                            badword = prebadword
                            badrule = rule
                            risklevel = probRisk[tolerance.index(prebadword)]
                                        
                pastword = word
                r = r+1

            if minW < 999:# and minW > 0:
                check = True
                denom = list(grouped[s]).count(word)
                t = -1
                k = -1
                num = 0

                certainty = round(probRisk[tolerance.index(badword)])
                softlim = limit
                for d in range(denom):
                    t = fulllist.find(word, t + 1)
                    k = fulllist.find(badword, t+1, t+softlim*size)
                    if k > -1:
                        num = num+1
                prob = round(100*num/denom)
                
                if prob > 0:
                    if risk == 0:
                        print(a,word,"WLO: State",badword,"with",certainty,'% anomaly risk, in the next',softlim,'words with',prob,'% probability')
                    if risk == 1:
                        fod = " "+" "*len(str(a)) + " "*len(word)
                        print(fod,"WLO: State",badword,"with",certainty,'% anomaly risk, in the next',softlim,'words with',prob,'% probability')
                elif prob == 0:
                    if risk == 0:
                        print(a,word,"WLO: State",badword,"with",certainty,'% anomaly risk, in the next',softlim,'words with minimal chance')
                    if risk == 1:
                        fod = " "+" "*len(str(a)) + " "*len(word)
                        print(fod,"WLO: State",badword,"with",certainty,'% anomaly risk, in the next',softlim,'words with minimal chance')
            else:
                if check:
                    badRuleCounter = [-1]*len(badRuleCounter)
                check = False
                if word not in tolerance:
                    print(a,word,'Network is healthy')

  

sym0
/home/alexcasar/Desktop/snet/gits/lang-learn-repo/alex_tests/data/data11/lang/sym0_PCA_3_3/dict.txt
81
['R10', 'R11', 'R13', 'R14', 'R15', 'R16', 'R19', 'R20', 'R21', 'R23', 'R24', 'R25', 'R26', 'R3', 'R4', 'R6']
1 R18 WLO: State R21 with 50 % anomaly risk, in the next 10 words with 100 % probability
2 R18 WLO: State R21 with 50 % anomaly risk, in the next 10 words with 100 % probability
3 R18 WLO: State R21 with 50 % anomaly risk, in the next 10 words with 100 % probability
4 R18 WLO: State R21 with 50 % anomaly risk, in the next 10 words with 100 % probability
5 R18 WLO: State R21 with 50 % anomaly risk, in the next 10 words with 100 % probability
6 R18 WLO: State R21 with 50 % anomaly risk, in the next 10 words with 100 % probability
7 R18 WLO: State R21 with 50 % anomaly risk, in the next 10 words with 100 % probability
8 R21 WHI: State R21 with 50 % chance of being anomaouls, currently active
      WLO: State R24 with 100 % anomaly risk, in the next 10 words with 90 % probabi

       WLO: State R15 with 100 % anomaly risk, in the next 10 words with minimal chance
277 R6 WHI: State R6 with 50 % chance of being anomaouls, currently active
       WLO: State R15 with 100 % anomaly risk, in the next 10 words with minimal chance
278 R6 WHI: State R6 with 50 % chance of being anomaouls, currently active
       WLO: State R15 with 100 % anomaly risk, in the next 10 words with minimal chance
279 R6 WHI: State R6 with 50 % chance of being anomaouls, currently active
       WLO: State R15 with 100 % anomaly risk, in the next 10 words with minimal chance
280 R6 WHI: State R6 with 50 % chance of being anomaouls, currently active
       WLO: State R15 with 100 % anomaly risk, in the next 10 words with minimal chance
281 R6 WHI: State R6 with 50 % chance of being anomaouls, currently active
       WLO: State R15 with 100 % anomaly risk, in the next 10 words with minimal chance
282 R6 WHI: State R6 with 50 % chance of being anomaouls, currently active
       WLO: State R15 

434 R3 WHI: State R3 with 50 % chance of being anomaouls, currently active
       WLO: State R4 with 100 % anomaly risk, in the next 10 words with 12 % probability
435 R3 WHI: State R3 with 50 % chance of being anomaouls, currently active
       WLO: State R4 with 100 % anomaly risk, in the next 10 words with 12 % probability
436 R3 WHI: State R3 with 50 % chance of being anomaouls, currently active
       WLO: State R4 with 100 % anomaly risk, in the next 10 words with 12 % probability
437 R3 WHI: State R3 with 50 % chance of being anomaouls, currently active
       WLO: State R4 with 100 % anomaly risk, in the next 10 words with 12 % probability
438 R3 WHI: State R3 with 50 % chance of being anomaouls, currently active
       WLO: State R4 with 100 % anomaly risk, in the next 10 words with 12 % probability
439 R3 WHI: State R3 with 50 % chance of being anomaouls, currently active
       WLO: State R4 with 100 % anomaly risk, in the next 10 words with 12 % probability
440 R3 WHI: Stat

        WLO: State R4 with 100 % anomaly risk, in the next 10 words with 16 % probability
625 R13 WHI: State R13 with 50 % chance of being anomaouls, currently active
        WLO: State R4 with 100 % anomaly risk, in the next 10 words with 16 % probability
626 R13 WHI: State R13 with 50 % chance of being anomaouls, currently active
        WLO: State R4 with 100 % anomaly risk, in the next 10 words with 16 % probability
627 R13 WHI: State R13 with 50 % chance of being anomaouls, currently active
        WLO: State R4 with 100 % anomaly risk, in the next 10 words with 16 % probability
628 R13 WHI: State R13 with 50 % chance of being anomaouls, currently active
        WLO: State R4 with 100 % anomaly risk, in the next 10 words with 16 % probability
629 R13 WHI: State R13 with 50 % chance of being anomaouls, currently active
        WLO: State R4 with 100 % anomaly risk, in the next 10 words with 16 % probability
630 R13 WHI: State R13 with 50 % chance of being anomaouls, currently activ

        WLO: State R19 with 100 % anomaly risk, in the next 10 words with 11 % probability
788 R20 WHI: State R20 with 50 % chance of being anomaouls, currently active
        WLO: State R19 with 100 % anomaly risk, in the next 10 words with 11 % probability
789 R20 WHI: State R20 with 50 % chance of being anomaouls, currently active
        WLO: State R19 with 100 % anomaly risk, in the next 10 words with 11 % probability
790 R20 WHI: State R20 with 50 % chance of being anomaouls, currently active
        WLO: State R19 with 100 % anomaly risk, in the next 10 words with 11 % probability
791 R20 WHI: State R20 with 50 % chance of being anomaouls, currently active
        WLO: State R19 with 100 % anomaly risk, in the next 10 words with 11 % probability
792 R20 WHI: State R20 with 50 % chance of being anomaouls, currently active
        WLO: State R19 with 100 % anomaly risk, in the next 10 words with 11 % probability
793 R20 WHI: State R20 with 50 % chance of being anomaouls, currently

...

       WLO: State R4 with 50 % anomaly risk, in the next 10 words with 38 % probability
103 R4 WHI: State R4 with 50 % chance of being anomaouls, currently active
       WLO: State R3 with 50 % anomaly risk, in the next 10 words with 11 % probability
104 R4 WHI: State R4 with 50 % chance of being anomaouls, currently active
       WLO: State R3 with 50 % anomaly risk, in the next 10 words with 11 % probability
105 R8 WHI: State R8 with 50 % chance of being anomaouls, currently active
       WLO: State R16 with 100 % anomaly risk, in the next 10 words with minimal chance
106 R8 WHI: State R8 with 50 % chance of being anomaouls, currently active
       WLO: State R16 with 100 % anomaly risk, in the next 10 words with minimal chance
107 R4 WHI: State R4 with 50 % chance of being anomaouls, currently active
       WLO: State R3 with 50 % anomaly risk, in the next 10 words with 11 % probability
108 R4 WHI: State R4 with 50 % chance of being anomaouls, currently active
       WLO: State R3 w

414 R7 WHI: State R7 with 50 % chance of being anomaouls, currently active
       WLO: State R4 with 50 % anomaly risk, in the next 10 words with 38 % probability
415 R6 WHI: State R6 with 50 % chance of being anomaouls, currently active
       WLO: State R16 with 100 % anomaly risk, in the next 10 words with minimal chance
416 R6 WHI: State R6 with 50 % chance of being anomaouls, currently active
       WLO: State R16 with 100 % anomaly risk, in the next 10 words with minimal chance
417 R7 WHI: State R7 with 50 % chance of being anomaouls, currently active
       WLO: State R4 with 50 % anomaly risk, in the next 10 words with 38 % probability
418 R8 WHI: State R8 with 50 % chance of being anomaouls, currently active
       WLO: State R16 with 100 % anomaly risk, in the next 10 words with minimal chance
419 R8 WHI: State R8 with 50 % chance of being anomaouls, currently active
       WLO: State R16 with 100 % anomaly risk, in the next 10 words with minimal chance
420 R6 WHI: State R6 w

585 R4 WHI: State R4 with 50 % chance of being anomaouls, currently active
       WLO: State R3 with 50 % anomaly risk, in the next 10 words with 11 % probability
586 R8 WHI: State R8 with 50 % chance of being anomaouls, currently active
       WLO: State R16 with 100 % anomaly risk, in the next 10 words with minimal chance
587 R8 WHI: State R8 with 50 % chance of being anomaouls, currently active
       WLO: State R16 with 100 % anomaly risk, in the next 10 words with minimal chance
588 R4 WHI: State R4 with 50 % chance of being anomaouls, currently active
       WLO: State R3 with 50 % anomaly risk, in the next 10 words with 11 % probability
589 R4 WHI: State R4 with 50 % chance of being anomaouls, currently active
       WLO: State R3 with 50 % anomaly risk, in the next 10 words with 11 % probability
590 R4 WHI: State R4 with 50 % chance of being anomaouls, currently active
       WLO: State R3 with 50 % anomaly risk, in the next 10 words with 11 % probability
591 R8 WHI: State R8 w

        WLO: State R16 with 100 % anomaly risk, in the next 10 words with 8 % probability
890 R15 WHI: State R15 with 50 % chance of being anomaouls, currently active
        WLO: State R16 with 100 % anomaly risk, in the next 10 words with 8 % probability
891 R15 WHI: State R15 with 50 % chance of being anomaouls, currently active
        WLO: State R16 with 100 % anomaly risk, in the next 10 words with 8 % probability
892 R15 WHI: State R15 with 50 % chance of being anomaouls, currently active
        WLO: State R16 with 100 % anomaly risk, in the next 10 words with 8 % probability
893 R15 WHI: State R15 with 50 % chance of being anomaouls, currently active
        WLO: State R16 with 100 % anomaly risk, in the next 10 words with 8 % probability
894 R17 WHI: State R17 with 50 % chance of being anomaouls, currently active
        WLO: State R16 with 100 % anomaly risk, in the next 10 words with 15 % probability
895 R15 WHI: State R15 with 50 % chance of being anomaouls, currently acti

         WLO: State R0 with 50 % anomaly risk, in the next 10 words with 21 % probability
1064 R9 WHI: State R9 with 50 % chance of being anomaouls, currently active
        WLO: State R0 with 50 % anomaly risk, in the next 10 words with 53 % probability
1065 R13 WHI: State R13 with 50 % chance of being anomaouls, currently active
         WLO: State R10 with 50 % anomaly risk, in the next 10 words with 38 % probability
1066 R9 WHI: State R9 with 50 % chance of being anomaouls, currently active
        WLO: State R0 with 50 % anomaly risk, in the next 10 words with 53 % probability
1067 R13 WHI: State R13 with 50 % chance of being anomaouls, currently active
         WLO: State R10 with 50 % anomaly risk, in the next 10 words with 38 % probability
1068 R0 WHI: State R0 with 50 % chance of being anomaouls, currently active
        WLO: State R0 with 50 % anomaly risk, in the next 10 words with 50 % probability
1069 R9 WHI: State R9 with 50 % chance of being anomaouls, currently active
 